# Imports

In [1]:
import pandas as pd
import numpy as np
import keras


Using TensorFlow backend.


# Load the dataset

In [5]:
# one hot encode the dataset, only take 6:00, 12:00 and 18:00 b/c that when most 
# of the times are taken
dataframe = pd.read_csv("atlantic.csv")
dataframe["time"] = dataframe["time"].astype('category')
dataframe = pd.get_dummies(dataframe, columns=["time","rating",])



y = dataframe[["rating_ DB","rating_ EX","rating_ HU","rating_ LO","rating_ SD","rating_ SS","rating_ TD","rating_ TS","rating_ WV"]].values
X = dataframe[["wind_speed","pressure", "time_600", "time_1200","time_1800"]].values


# Clean the data

In [6]:
# delete all the rows that contain no readings for pressure
rows_to_delete = []
row_num = 0
for row in X:
    if row[1] == -999:
        rows_to_delete.append(row_num)
    row_num += 1
X = np.delete(X, rows_to_delete, 0)
y = np.delete(y, rows_to_delete, 0)


# Split the dataset into training set and test set

In [7]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Train the model on training set

In [9]:
from keras.models import Sequential
from keras.layers import Dense

In [11]:
classifier = Sequential()

# input layer
classifier.add(Dense(output_dim=6, init='uniform', activation='relu', input_dim=5))

# hidden layer
classifier.add(Dense(output_dim=6, init='uniform', activation='relu'))

# output layer
classifier.add(Dense(output_dim=9, init='uniform', activation='softmax'))

# create the NN
classifier.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

# Predict the test set


In [12]:
classifier.fit(x=X_train, y=y_train, batch_size=10, epochs=10)


Epoch 1/10
12380/12380 [==============================] - 3s 217us/step - loss: 1.2160 - accuracy: 0.5919
Epoch 2/10
12380/12380 [==============================] - 3s 207us/step - loss: 0.8020 - accuracy: 0.7732
Epoch 3/10
12380/12380 [==============================] - 2s 197us/step - loss: 0.7727 - accuracy: 0.7838
Epoch 4/10
12380/12380 [==============================] - 3s 214us/step - loss: 0.7634 - accuracy: 0.7830
Epoch 5/10
12380/12380 [==============================] - 2s 193us/step - loss: 0.7457 - accuracy: 0.7847
Epoch 6/10
12380/12380 [==============================] - 3s 228us/step - loss: 0.7369 - accuracy: 0.7844
Epoch 7/10
12380/12380 [==============================] - 2s 197us/step - loss: 0.7290 - accuracy: 0.7855
Epoch 8/10
12380/12380 [==============================] - 3s 229us/step - loss: 0.7223 - accuracy: 0.7863
Epoch 9/10
12380/12380 [==============================] - 3s 222us/step - loss: 0.7179 - accuracy: 0.7827
Epoch 10/10
12380/12380 [====================

In [15]:
y_pred = classifier.predict(X_test)
# y_pred = (y_pred > 0.5)

In [53]:
for row in y_pred[:2]:
    most_likely = 0
    for el in row:
        if el > most_likely:
            most_likely = el

    print( "rating_DB: {DB} \nrating_EX: {EX}\nrating_HU: {HU} \nrating_LO: {LO} \nrating_SD: {SD} \nrating_SS: {SS} \nrating_TD: {TD} \nrating_TS: {TS} \nrating_WV: {WV} \n\nMost Likely: {ml}\n\n".format(DB=row[0], EX=row[1], HU=row[2], LO=row[3], SD=row[4], SS=row[5], TD=row[6], TS=row[7], WV=row[8], ml=most_likely) )

rating_DB: 0.008576853200793266 
rating_EX: 0.0668913722038269
rating_HU: 5.985428500474775e-26 
rating_LO: 0.2972959280014038 
rating_SD: 0.07645807415246964 
rating_SS: 0.0004922611988149583 
rating_TD: 0.5311508178710938 
rating_TS: 0.007046234328299761 
rating_WV: 0.012088434770703316 

Most Likely: 0.5311508178710938


rating_DB: 0.0005689025274477899 
rating_EX: 0.23434194922447205
rating_HU: 1.795838358959083e-16 
rating_LO: 0.01000883337110281 
rating_SD: 3.030112793567241e-06 
rating_SS: 0.039577312767505646 
rating_TD: 8.545827768102754e-06 
rating_TS: 0.7149816155433655 
rating_WV: 0.0005097754765301943 

Most Likely: 0.7149816155433655


